In [1]:
import awkward as ak
import numpy as np
import numba as nb
import hist
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
from common import load_events, DelphesSchema

In [2]:
class DelphesSchema2(DelphesSchema):
    def __init__(self, base_form):
        for key in list(base_form["contents"].keys()):
            if "fBits" in key:
                base_form["contents"].pop(key, None)
        super().__init__(base_form)

In [3]:
# load data
events = load_events(
    "models/s-channel_mmed-1000_Nc-2_Nf-2_scale-35.1539_mq-10_mpi-20_mrho-20_pvector-0.75_spectrum-cms_rinv-0.3/events.root",
    schema=DelphesSchema2,
)

In [4]:
@nb.njit
def get_constituents_kernel(jets, cands):
    # get hash table mapping global index : global unique ID
    hash_table = {k:v for v,k in enumerate(cands.fUniqueID)}

    output = []
    for jet in jets:
        # apply hash map
        translated = [hash_table[ref] for ref in jet.Constituents.refs]
        output.append(translated)

    return output

def get_constituents(events, jetsname, candsname):
    output = None
    ctr = 0
    for jets,cands in zip(events[jetsname], events[candsname]):
        #print(ctr)
        #ctr+=1
        indices = get_constituents_kernel(jets, cands) if jets is not None else []
        if len(indices)==0:
            unflattened = None
        else:
            unflattened = ak.unflatten(cands[ak.flatten(indices)],ak.count(indices,axis=1),behavior=cands.behavior)[None]
        if output is None:
            output = unflattened # add first dimension
        else:
            output = ak.concatenate([output,unflattened])
    return ak.with_name(output, "Particle")

In [5]:
%%timeit -n 1 -r 1
events["FatJet","ConstituentsFull"] = get_constituents(events,"FatJet","ParticleFlowCandidate")

/tmp/ipykernel_2600354/121641287.py:9: NumbaTypeSafetyWarning: unsafe cast from int64 to uint32. Precision may be lost.
  translated = [hash_table[ref] for ref in jet.Constituents.refs]


2min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
def sum_4vec(vec):
    summed_vec = {
        "t": ak.sum(vec.E,axis=-1),
        "x": ak.sum(vec.px,axis=-1),
        "y": ak.sum(vec.py,axis=-1),
        "z": ak.sum(vec.pz,axis=-1),
    }
    return ak.zip(summed_vec,with_name="LorentzVector")
check_jets = sum_4vec(events.FatJet.ConstituentsFull)
print(check_jets.mass-events.FatJet.mass)

[[0.000565, 1.43e-06, -3.81e-06], [1.91e-05, ... [0.000404, 6.1e-05, 0.000409]]
